<a href="https://colab.research.google.com/github/NesquikPlz/kisa_rugpullpredection_xgboost/blob/main/kisa_rugpullpredetection_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###hi

In [15]:
import xgboost as xgb ## XGBoost 불러오기
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance ## Feature Importance를 불러오기 위함
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split

# import warnings
# warnings.filterwarnings('ignore')

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4


In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
train = pd.read_csv('/content/drive/MyDrive/dataset/Dataset_v1.0.csv')
train

,id,token00.id,token00.name,is_rugpull,txCount,mint_count,swap_count,burn_count,mint_ratio,swap_ratio,burn_ratio,mint_mean_period,swap_mean_period,burn_mean_period,swapIn,swapOut,swap_rate,active_period,해당 시점에 LP제공자의 유동성 풀 보유 비율,락 비율,Token_burn_rate,개발자 토큰 보유 비율,개발자가 만든 토큰의 수
0,0x117cc9c215505f8c1fe18822d4227f86f861fdbf,0x91276bb077e18d93b93a471160a65c7bc5d8ad72,Shoko Inu,True,88,1,87,0,0.011364,0.988636,0.000000,0.000000,0.148506,0.000000,59,28,2.034483,5488,1.00,0.0,0.01,0.08,1
1,0x209582ff421988d1f74b66742fa94fc533e3b18c,0xbbac15e551f4d086453f0304aecf4fadf03f9885,Sapporo Shiba,True,27,1,31,0,0.031250,0.968750,0.000000,0.000000,0.164714,0.000000,26,5,4.333333,33039,0.00,1.0,0.00,0.00,1
2,0x2385ba58d184eaed27a46b758bfa86fdf3dfd96e,0xdaa1e2fd1737026a451a330d8caee031f0d2a1c8,Kanye Inu,True,85,1,86,0,0.011494,0.988506,0.000000,0.000000,0.101054,0.000000,56,30,1.806452,16793,1.00,0.0,0.00,0.08,1
3,0x2f59dfcfa75f0234bfbf8c0cc570500dd3078ffd,0xa2e3132cf93e2197c10cdf2db4ee5a0653f292c0,nano SAITAMA,True,73,1,75,0,0.013158,0.986842,0.000000,0.000000,0.075108,0.000000,37,38,0.948718,15511,1.00,0.0,0.00,0.11,1
4,0x46a9aa15cb99755fae5caa55890f509cfa33fd8a,0xc9d29e447021eedfd565e1ac38d61d1a02d5fc27,ButtCoin,True,9,1,7,0,0.125000,0.875000,0.000000,0.000000,0.287891,0.000000,4,3,1.000000,30185,1.00,0.0,0.00,0.10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17880,0xc5be99a02c6857f9eac67bbce58df5572498f40c,0xd46ba6d942050d489dbd938a2c909a5d5039a161,Ampleforth,False,395640,14,15,4,0.424242,0.454545,0.121212,0.331564,0.595371,0.585596,6,9,0.600000,584331,0.00,0.0,0.00,0.00,1
17881,0x6aeebc2f5c979fd5c4361c2d288e55ac6b7e39bb,0x1beef31946fbbb40b877a72e4ae04a8d1a5cee06,Parachute,False,30626,8,95,0,0.077670,0.922330,0.000000,0.197950,0.356471,0.000000,56,39,1.400000,586531,0.06,0.0,0.00,0.00,1
17882,0x9b533f1ceaa5ceb7e5b8994ef16499e47a66312d,0x4575f41308ec1483f3d399aa9a2826d74da13deb,Orchid,False,19218,4,33,0,0.108108,0.891892,0.000000,0.237250,0.498648,0.000000,19,14,1.266667,594660,0.01,0.0,0.00,0.00,1
17883,0x2b6a25f7c54f43c71c743e627f5663232586c39f,0x8a9c67fee641579deba04928c4bc45f66e26343a,Jarvis Reward Token,False,24213,12,212,8,0.051724,0.913793,0.034483,0.249655,0.309396,0.586083,119,93,1.265957,597043,0.00,0.0,0.00,0.10,1


In [44]:
trainset = train.drop(columns = ['token00.id', 'token00.name'])
trainset['is_rugpull'] = trainset['is_rugpull'].replace({True : 1, False : 0})
target = 'is_rugpull'
IDcol = 'id'
trainset

,id,is_rugpull,txCount,mint_count,swap_count,burn_count,mint_ratio,swap_ratio,burn_ratio,mint_mean_period,swap_mean_period,burn_mean_period,swapIn,swapOut,swap_rate,active_period,해당 시점에 LP제공자의 유동성 풀 보유 비율,락 비율,Token_burn_rate,개발자 토큰 보유 비율,개발자가 만든 토큰의 수
0,0x117cc9c215505f8c1fe18822d4227f86f861fdbf,1,88,1,87,0,0.011364,0.988636,0.000000,0.000000,0.148506,0.000000,59,28,2.034483,5488,1.00,0.0,0.01,0.08,1
1,0x209582ff421988d1f74b66742fa94fc533e3b18c,1,27,1,31,0,0.031250,0.968750,0.000000,0.000000,0.164714,0.000000,26,5,4.333333,33039,0.00,1.0,0.00,0.00,1
2,0x2385ba58d184eaed27a46b758bfa86fdf3dfd96e,1,85,1,86,0,0.011494,0.988506,0.000000,0.000000,0.101054,0.000000,56,30,1.806452,16793,1.00,0.0,0.00,0.08,1
3,0x2f59dfcfa75f0234bfbf8c0cc570500dd3078ffd,1,73,1,75,0,0.013158,0.986842,0.000000,0.000000,0.075108,0.000000,37,38,0.948718,15511,1.00,0.0,0.00,0.11,1
4,0x46a9aa15cb99755fae5caa55890f509cfa33fd8a,1,9,1,7,0,0.125000,0.875000,0.000000,0.000000,0.287891,0.000000,4,3,1.000000,30185,1.00,0.0,0.00,0.10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17880,0xc5be99a02c6857f9eac67bbce58df5572498f40c,0,395640,14,15,4,0.424242,0.454545,0.121212,0.331564,0.595371,0.585596,6,9,0.600000,584331,0.00,0.0,0.00,0.00,1
17881,0x6aeebc2f5c979fd5c4361c2d288e55ac6b7e39bb,0,30626,8,95,0,0.077670,0.922330,0.000000,0.197950,0.356471,0.000000,56,39,1.400000,586531,0.06,0.0,0.00,0.00,1
17882,0x9b533f1ceaa5ceb7e5b8994ef16499e47a66312d,0,19218,4,33,0,0.108108,0.891892,0.000000,0.237250,0.498648,0.000000,19,14,1.266667,594660,0.01,0.0,0.00,0.00,1
17883,0x2b6a25f7c54f43c71c743e627f5663232586c39f,0,24213,12,212,8,0.051724,0.913793,0.034483,0.249655,0.309396,0.586083,119,93,1.265957,597043,0.00,0.0,0.00,0.10,1


In [45]:
trainset = trainset.dropna(how='any',axis = 0)
trainset.isnull().sum()

id                           0
is_rugpull                   0
txCount                      0
mint_count                   0
swap_count                   0
burn_count                   0
mint_ratio                   0
swap_ratio                   0
burn_ratio                   0
mint_mean_period             0
swap_mean_period             0
burn_mean_period             0
swapIn                       0
swapOut                      0
swap_rate                    0
active_period                0
해당 시점에 LP제공자의 유동성 풀 보유 비율    0
락 비율                         0
Token_burn_rate              0
개발자 토큰 보유 비율                 0
개발자가 만든 토큰의 수                0
dtype: int64

In [46]:
trainset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17793 entries, 0 to 17884
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         17793 non-null  object 
 1   is_rugpull                 17793 non-null  int64  
 2   txCount                    17793 non-null  int64  
 3   mint_count                 17793 non-null  int64  
 4   swap_count                 17793 non-null  int64  
 5   burn_count                 17793 non-null  int64  
 6   mint_ratio                 17793 non-null  float64
 7   swap_ratio                 17793 non-null  float64
 8   burn_ratio                 17793 non-null  float64
 9   mint_mean_period           17793 non-null  float64
 10  swap_mean_period           17793 non-null  float64
 11  burn_mean_period           17793 non-null  float64
 12  swapIn                     17793 non-null  int64  
 13  swapOut                    17793 non-null  int

In [49]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['is_rugpull'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(dtrain['is_rugpull'].values, dtrain_predictions))
    print("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['is_rugpull'], dtrain_predprob))
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [50]:
#Choose all predictors except target & IDcols
predictors = [x for x in trainset.columns if x not in [target, IDcol]]
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, trainset, predictors)


Model Report


NameError: ignored